In [ ]:
import arcgis
from arcgis.gis import GIS
import re, csv
import pandas as pd
# Define variables
portalURL = r'https://gissp.bracell.com/portal/'
username = #Inserir login do Survey que publicou a pesquisa
password = #Inserir senha do Survey que publicou a pesquisa
survey_item_id = #Inserir Id da pesquisa Survey
save_path = r"F:\Qualidade_Florestal\01- SÃO PAULO\04- Logística, transporte e estradas\00 - Bases de Trabalho"


keep_org_item = False
store_csv_w_attachments = False

# Connect to GIS Portal and identify Survey form
gis = GIS(portalURL, username, password)
survey_by_id = gis.content.get(survey_item_id)
print(survey_by_id.type)
survey_by_id

# Download service
rel_fs = survey_by_id.related_items('Survey2Service','forward')[0]
rel_fs

item_excel = rel_fs.export(title=survey_by_id.title, export_format='Excel')
item_excel.download(save_path=save_path)
if not bool(keep_org_item):
    item_excel.delete(force=True)

# Process feature layers and attachments
layers = rel_fs.layers + rel_fs.tables
for i in layers:
    if i.properties.hasAttachments:
        print("Item: ", i, "HAS ATTACHMENTS")
        feature_layer_folder = os.path.join(save_path, '{}_attachments'.format(re.sub(r'[^A-Za-z0-9]+', '', i.properties.name)))
        if not os.path.exists(feature_layer_folder):
            os.mkdir(feature_layer_folder)         
        if bool(store_csv_w_attachments):
            path = os.path.join(feature_layer_folder, "{}_attachments.csv".format(i.properties.name))
        elif not bool(store_csv_w_attachments):
            path = os.path.join(save_path, "{}_attachments.csv".format(i.properties.name))
        csv_fields = ['Parent objectId','Attachment path']
        with open(path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(csv_fields)
            feature_object_ids = i.query(where="1=1", return_ids_only=True, order_by_fields='objectid ASC')
            for j in range(len(feature_object_ids['objectIds'])):
                current_oid = feature_object_ids['objectIds'][j]
                current_oid_attachments = i.attachments.get_list(current_oid)
                if current_oid_attachments is not None and len(current_oid_attachments) > 0:
                    for k in range(len(current_oid_attachments)):
                        attachment_id = current_oid_attachments[k]['id']
                        global_id = str(i.query(where="1=1", return_geometry=False, return_ids_only=True, order_by_fields='objectid ASC')['objectIds'][j])
                        current_attachment_path = i.attachments.download(oid=current_oid, attachment_id=attachment_id, save_path=feature_layer_folder)
                        if current_attachment_path is not None:
                            new_attachment_path = os.path.join(feature_layer_folder, global_id + "_" + os.path.split(current_attachment_path[0])[1])
                            shutil.move(current_attachment_path[0], new_attachment_path)
                            csvwriter.writerow([current_oid, os.path.join('{}_attachments'.format(re.sub(r'[^A-Za-z0-9]+', '', i.properties.name)), os.path.split(new_attachment_path)[1])])
                        else:
                            print("Item: ", i, "DOES NOT HAVE ATTACHMENTS")
print('Done!')
            


# Ao final, as mensagens indicarão as respostas dos formulários que não tem foto e quando tiver concluído.

In [ ]:
#inserir caminho da base onde há os nomes dos campos para renomear as fotos
tabela = pd.read_excel(r"F:\Qualidade_Florestal\01- SÃO PAULO\04- Logística, transporte e estradas\00 - Bases de Trabalho\base.xlsx")


import imghdr


#Inserir caminho da base com os ids das avaliações
caminho_completo = os.path.abspath(r"F:\Qualidade_Florestal\01- SÃO PAULO\04- Logística, transporte e estradas\00 - Bases de Trabalho\base_avaliacao.xlsx")

for imagem in os.listdir(caminho_completo):
    if imghdr.what(os.path.join(caminho_completo, imagem)):
        for index, objectid in enumerate(tabela["objectid"]):
            if str(objectid) == str(imagem).split("_")[0]:
                novo_nome = str(tabela["empresa"][index]) + " - " + imagem
                if "motorista" in str(imagem.lower()):
                    novo_nome = str(tabela["matricula_motorista"][index]) + " - " + novo_nome
                if "operador" in str(imagem.lower()):
                    novo_nome = str(tabela["matricula_operador"][index]) + " - " + novo_nome
                if "malhal" in str(imagem.lower()) or "paralama" in str(imagem.lower()) or "fueiro" in str(imagem.lower()) or "cinta" in str(imagem.lower()):
                    novo_nome = str(tabela["empresa"][index]) + " - " + str(tabela["placa_caminhao"][index]).upper() + " - " + imagem
                novo_nome = novo_nome.replace("/", "-")  # substitui o ponto por hífen
                try:
                    shutil.move(os.path.join(caminho_completo, imagem), os.path.join(caminho_completo, novo_nome))
                except FileNotFoundError:
                    print(f"O arquivo {imagem} não foi encontrado")
                break
    else:
        print(f"O arquivo {imagem} não é uma imagem")